In [25]:
import pandas as pd
import numpy as np
from tpulse import TinkoffPulse
from time import sleep
from pytz import timezone
from httpx import HTTPStatusError
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import re

import time
from requests.exceptions import ReadTimeout

import warnings
warnings.filterwarnings('ignore')

import requests
from datetime import datetime, timedelta
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, Table, MetaData, insert, text, TIMESTAMP
from sqlalchemy.dialects.postgresql import insert
import requests


def connection():
    db_params = {
        'dbname': 'russian-stocks-prediction-ml-dl',
        'user': 'root',
        'password': 'groot',
        'host': '185.70.105.233',
        'port': '5432'
    }
    conn_str = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'
    return create_engine(conn_str)

In [26]:
full_df = pd.DataFrame(columns=['id', 'inserted', 'commentsCount', 'text', 'reactionCount','reactions_counters', 'ticker'])

d1 = pd.read_csv('/kaggle/input/hse-proj/first_csv.csv')
d2 = pd.read_csv('/kaggle/input/hse-proj/second_csv.csv')
d3 = pd.read_csv('/kaggle/input/hse-proj/third_csv.csv')

d4 = pd.read_csv('/kaggle/input/hse-proj/shares_1.csv')
d4 = d4[d4['ticker'].isin(["RASP", "RAGR", "MRKP", "SVAV", "AQUA", "RENI", "OZPH", "FIXR", "OGKB", "LEAS", "CBOM", "VSMO", "IRKT", "VSEH"])]

d5 = pd.read_csv('/kaggle/input/hse-proj/shares_2.csv')
d5 = d5[d5['ticker'].isin(["MRKC", "MDMG", "FLOT", "UPRO", "ASTR", "IVAT", "LENT", "WUSH", "UWGN", "ENPG", "TRMK", "MVID", "NMTP", "SELG",  "SOFL", "SFIN"])]

d6 = pd.read_csv('/kaggle/input/hse-proj/shares_3.csv')
d6 = d6[d6['ticker'].isin(['AFKS', 'AFLT', 'ALRS', 'BSPB', 'CHMF', 'EUTR', 'FEES', 'GMKN', 'HYDR', 'IRAO', 'LKOH', 'MAGN', 'MGNT', 'MOEX', 'MTLR', 'MTSS', 'NLMK', 'NVTK'])]

d7 = pd.read_csv('/kaggle/input/hse-proj/shares_4.csv')

d8 = pd.read_csv('/kaggle/input/hse-proj-1/shares_5.csv')

full_df = pd.concat([full_df, d1, d2, d3, d4, d5, d6, d7, d8], axis=0)

In [37]:
columns_order = ['id', 'ticker', 'inserted', 'text', 'commentsCount', 'reactionCount', 'reactions_counters']
full_df[['commentsCount', 'reactionCount']] = full_df[['commentsCount', 'reactionCount']].astype(int)
full_df['inserted'] = pd.to_datetime(full_df['inserted'])
full_df.head()

,id,inserted,commentsCount,text,reactionCount,reactions_counters,ticker
0,99dc28fa-2a2e-45e9-a374-6a8aea387d10,2025-10-30,11,Пришел рекордный кэшбек с начала года!\n\nПока...,9,"[{'type': 'like', 'count': 5}, {'type': 'buy-u...",BELU
1,1e7e0758-07a1-4090-99a1-8888ceb5baed,2025-10-30,4,{$BELU} ММ убирай плитки че не видиш она хоче...,0,[],BELU
2,621e12d3-af13-43f3-9a5c-275a2ac75a84,2025-10-30,30,{$BELU}\nЭмитент сегодня (30.10.2025 года) пер...,74,"[{'type': 'like', 'count': 67}, {'type': 'rock...",BELU
3,80d5b424-3348-4ab3-97b7-6dbc2818dffa,2025-10-30,7,{$BELU} ребят подскажите не особо шарящиму пож...,1,"[{'type': 'like', 'count': 1}]",BELU
4,631f5bd3-7ee8-4383-a2ab-3ca15121ab7d,2025-10-30,3,"{$BELU} слаба пока, алкогольная старушка: как ...",0,[],BELU


In [39]:
full_df[columns_order].to_sql('t_pulse_data', con=connection(), if_exists='replace', index=False)

251